In [1]:
import datetime
import Image
import gc
import numpy as np
import os
import random
from scipy import misc
import string
import time

# Set some Theano config before initializing
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=cpu,floatX=float32,allow_gc=False,openmp=True"
import theano

import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt

#import emotion_model
import dwdii_bc_model_helper as bc
import bc_models as models

random.seed(20275)

Using Theano backend.


In [2]:
print "device:", theano.config.device
print "floatX:",  theano.config.floatX
print "mode:", theano.config.mode
print "openmp:", theano.config.openmp
print "allow_gc:", theano.config.allow_gc

device: cpu
floatX: float32
mode: FAST_RUN
openmp: True
allow_gc: False


In [3]:
imagePath = "/root/bc_data/ddsm-png.25"
trainDataPath = "../../data/ddsm_train.csv"
testDataPath = "../../data/ddsm_test.csv"
imgResize = (150, 150)

In [4]:
os.listdir('../../data')

['ddsm_test.csv', 'ddsm_train.csv', 'ddsm_val.csv']

# Load Training and Test Data

In this section, the training/validation data is loaded. The load_data function pre-balances the data set by removing images from over-represented emotion classes.

### Training Data

In [6]:
metaData, meta2, mCounts = bc.load_training_metadata(trainDataPath, balanceViaRemoval=True, verbose=True)

Raw Balance
----------------
benign 541
malignant 852
normal 2793
balanaceViaRemoval.avgE: 1395
balanaceViaRemoval.theshold: 1395.0

After Balancing
----------------
benign 541
malignant 852
normal 929


In [7]:
meta2[meta2.keys()[0]]

'0'

In [8]:
# Actually load some representative data for model experimentation
maxData = len(metaData)
X_data, Y_data = bc.load_data(trainDataPath, imagePath, maxData = maxData, verboseFreq = 50, imgResize=imgResize)
print X_data.shape
print Y_data.shape

Raw Balance
----------------
benign 541
malignant 852
normal 2793
balanaceViaRemoval.avgE: 1395
balanaceViaRemoval.theshold: 1395.0

After Balancing
----------------
benign 541
malignant 852
normal 929
0.000000: C_0418_1.LEFT_CC.LJPEG.png
0.021533: C_0160_1.LEFT_MLO.LJPEG.png
0.043066: A_0462_1.LEFT_MLO.LJPEG.png
0.064599: C_0343_1.LEFT_CC.LJPEG.png
0.086133: B_3651_1.LEFT_CC.LJPEG.png
0.107666: C_0123_1.LEFT_CC.LJPEG.png
0.129199: B_3059_1.LEFT_MLO.LJPEG.png
0.150732: A_0578_1.LEFT_MLO.LJPEG.png
0.172265: C_0190_1.RIGHT_CC.LJPEG.png
0.193798: B_3398_1.LEFT_CC.LJPEG.png
0.215332: A_0325_1.RIGHT_CC.LJPEG.png
0.236865: C_0041_1.RIGHT_CC.LJPEG.png
0.258398: C_0046_1.RIGHT_CC.LJPEG.png
0.279931: B_3006_1.LEFT_CC.LJPEG.png
0.301464: A_1042_1.LEFT_MLO.LJPEG.png
0.322997: B_3011_1.LEFT_CC.LJPEG.png
0.344531: A_1104_1.LEFT_MLO.LJPEG.png
0.366064: B_3627_1.LEFT_MLO.LJPEG.png
0.387597: C_0005_1.RIGHT_MLO.LJPEG.png
0.409130: A_1035_1.RIGHT_CC.LJPEG.png
0.430663: A_0009_1.RIGHT_MLO.LJPEG.png
0.452

### Load Test Set 

In [9]:
# Actually load some representative data for model experimentation
maxData = len(metaData)
X_test, Y_test = bc.load_data(testDataPath, imagePath, maxData = maxData, verboseFreq = 50, imgResize=imgResize)
print X_test.shape
print Y_test.shape

Raw Balance
----------------
benign 142
malignant 206
normal 699
balanaceViaRemoval.avgE: 349
balanaceViaRemoval.theshold: 349.0

After Balancing
----------------
benign 142
malignant 206
normal 232
0.000000: B_3380_1.RIGHT_MLO.LJPEG.png
0.021533: C_0114_1.LEFT_CC.LJPEG.png
0.043066: C_0005_1.LEFT_MLO.LJPEG.png
0.064599: A_1032_1.RIGHT_CC.LJPEG.png
0.086133: B_3504_1.RIGHT_CC.LJPEG.png
0.107666: A_1102_1.RIGHT_MLO.LJPEG.png
0.129199: A_0149_1.LEFT_CC.LJPEG.png
0.150732: B_3039_1.LEFT_CC.LJPEG.png
(1554, 150, 150)
(1554, 1)


## Split Training/Test Sets
The following code segment splits the data into training and test data sets. Currently this is a standard 80/20 split for training and test respectively after performing a random shuffle using the unison_shuffled_copies help method.

In [10]:
X_train = X_data
Y_train = Y_data

In [11]:
print X_train.shape
print X_test.shape

print Y_train.shape
print Y_test.shape

(1554, 150, 150)
(363, 150, 150)
(1554, 1)
(363, 1)


In [12]:
import collections
def yDist(y):
    bcCounts = collections.defaultdict(int)
    for a in range(0, y.shape[0]):
        bcCounts[y[a][0]] += 1
    return bcCounts

print "Y_train Dist: " + str(yDist(Y_train))
print "Y_test Dist: " + str(yDist(Y_test))


Y_train Dist: defaultdict(<type 'int'>, {0: 765, 1: 50, 2: 739})
Y_test Dist: defaultdict(<type 'int'>, {0: 172, 1: 12, 2: 179})


## Define and Load Trained Model

In [13]:
# Load the bc array for our count in the model definition
bcTypes = bc.bcNumerics()
print bcTypes
print len(bcTypes)

{'benign': 1, 'malignant': 2, 'normal': 0}
3


In [14]:
# Construct the model using our help function
model = models.bc_model_v0(len(bcTypes), verbose=True, 
                                        input_shape=(1,X_train.shape[1],X_train.shape[2]))

____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
convolution2d_1 (Convolution2D)    (None, 32, 143, 143)2080        convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)          (None, 32, 143, 143)0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)      (None, 32, 71, 71)  0           activation_1[0][0]               
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)    (None, 32, 67, 67)  25632       maxpooling2d_1[0][0]             
___________________________________________________________________________________________

## Training the Model

The following code segment trains the model using the run_network helper function. 

In [15]:
loadWeights = False
if loadWeights:
    model.load_weights("dwdii-bc-150-v0-Cloud.hdf5")

In [16]:
# Reshape to the appropriate shape for the CNN input
testX = X_test.reshape(X_test.shape[0], 1, X_test.shape[1],X_test.shape[2])
trainX = X_train.reshape(X_train.shape[0], 1, X_train.shape[1],X_train.shape[2])

In [17]:
print "Training start: " + str(datetime.datetime.now())
m, h = models.run_network([trainX, testX, Y_train, Y_test], model, batch=50, epochs=30, verbosity=1)

Training start: 2017-03-28 01:37:04.157127
(1554, 3)
(363, 3)
Training model...
Train on 1554 samples, validate on 363 samples
Epoch 1/30
1554/1554 [==============================] - 299s - loss: 0.5670 - acc: 0.5650 - val_loss: 0.5046 - val_acc: 0.5482
Epoch 2/30
1554/1554 [==============================] - 297s - loss: 0.4692 - acc: 0.6293 - val_loss: 0.6298 - val_acc: 0.4986
Epoch 3/30
1554/1554 [==============================] - 317s - loss: 0.4463 - acc: 0.6699 - val_loss: 0.4648 - val_acc: 0.6694
Epoch 4/30
1554/1554 [==============================] - 328s - loss: 0.4487 - acc: 0.6557 - val_loss: 0.4848 - val_acc: 0.6198
Epoch 5/30
1554/1554 [==============================] - 322s - loss: 0.4218 - acc: 0.6898 - val_loss: 0.9780 - val_acc: 0.4821
Epoch 6/30
1554/1554 [==============================] - 286s - loss: 0.4252 - acc: 0.7066 - val_loss: 0.5667 - val_acc: 0.5758
Epoch 7/30
1554/1554 [==============================] - 276s - loss: 0.3986 - acc: 0.7169 - val_loss: 0.5349 - 

In [18]:
model.save_weights("dwdii-bc-150-v0-Cloud-20170327.hdf5", overwrite=True)

### Experiment Results

#### Raw DDSM Images

Initial results based on "normal" being masked as "benign":
* bc_model_v0 (150x150, 800/200): 182s - loss: 0.0560 - acc: 0.9813 - val_loss: 1.9918 - val_acc: 0.6800
* bc_model_v0 (150x150, 2000/500): 473s - loss: 0.0288 - acc: 0.9925 - val_loss: 1.4040 - val_acc: 0.7260
   * somewhat balanced, Y_train Dist {0: 1223, 1: 777}, Y_test Dist: {0: 321, 1: 179}

Revised with "normal", "benign" and "malignant" labeled seperately:
* bc_model_v0 (150x150, 1311/328): 298s - loss: 0.0411 - acc: 0.9786 - val_loss: 1.3713 - val_acc: 0.6616

After creating fixed "train", "test" and "validate" data sets, using "train" and "test" as well as including the DDSM Benign cases:
* bc_model_v0 (150x150, 1554/363, 03.27.2017): 264s - loss: 0.0512 - acc: 0.9730 - val_loss: 1.3120 - val_acc: 0.6116

In [17]:
661 / 1311.0

0.5041952707856598

In [18]:
167.0 / 328

0.5091463414634146